In [19]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from datetime import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from data import dataclass
from typing import List

In [ ]:
class BaseClass(dataclass):

    def __init__(self,
    tickers: List[str] = ['aapl'],
    feature_steps: int = 10,
    target_steps: int = 1,
    ):
        super().__init__()
        self.tickers = tickers
        self.BG = dict()
        self.dates = dict()
        self.df = dict()
        for t in self.tickers:
            loc = self.tickersloc[t]
            BG = self.BGP[loc[0]]["parms"][self.BGP[loc[0]]["ticker"][loc[1]]]
            d = str(int(BG[0][0]))
            self.BG[t] = BG[:][1:-1]
            self.df[t] = pd.DataFrame(np.transpose(self.BG[t]), columns = ["bp","cp","bn","cn"])
            self.dates[t] = [str(int(d)) for d in BG[:][0]]
            self.dates[t] = [dt.strptime(d, '%Y%m%d') for d in self.dates[t]]
        self.feature_steps = feature_steps
        self.target_steps = target_steps
        self.ts = {}
        self.X = {}
        self.y = {}
        self.X_train_full = {}
        self.y_train_full = {}
        self.X_test = {}
        self.y_test = {}
        self.X_train = {}
        self.y_train = {}
        self.X_valid = {}
        self.y_valid = {}
        self.split_ind = {}
        self.split_ind_2 = {}
        self.prc = {}
        self.scalers = {}
        self.test_pred_rescaled = {}
        for t in self.tickers:
            data = self.df[t].diff().dropna()
            self.ts[t] = data.values
            self.X[t], self.y[t] = self.ts_split(self.ts[t])
            self.split_ind[t] = int(self.X[t].shape[0]*0.8)
            self.X_train_full[t], self.y_train_full[t] = self.X[t][:self.split_ind[t]], self.y[t][:self.split_ind[t]]
            self.X_test[t], self.y_test[t] = self.X[t][self.split_ind[t]:], self.y[t][self.split_ind[t]:]
            self.split_ind_2[t] = int(self.X_train_full[t].shape[0]*0.8)
            self.X_train[t], self.y_train[t] = self.X_train_full[t][:self.split_ind_2[t]], self.y_train_full[t][:self.split_ind_2[t]]
            self.X_valid[t], self.y_valid[t] = self.X_train_full[t][self.split_ind_2[t]:], self.y_train_full[t][self.split_ind_2[t]:]
    
    def ts_split(self,
        ts
    ):
        feature_steps = self.feature_steps
        target_steps = self.target_steps
        n_obs = len(ts) - feature_steps - target_steps + 1
        X = np.array([ts[idx:idx + feature_steps] for idx in range(n_obs)])
        y = np.array([ts[idx + feature_steps:idx + feature_steps + target_steps][:, -1]
                    for idx in range(n_obs)])
        return X, y

In [99]:
Data = dataclass()

In [100]:
Base = BaseClass(tickers = ['a'])

In [102]:
t = 'a'
Base.df[t]
#Base.df[t]

,bp,cp,bn,cn
0,0.008234,1.524451,0.007213,1.524446
1,0.006954,1.703079,0.010747,1.029480
2,0.008235,1.555654,0.007228,1.555648
3,0.008241,1.553773,0.007232,1.553766
4,0.008101,1.599773,0.007109,1.599770
...,...,...,...,...
3268,0.011348,1.294109,0.021605,0.636920
3269,0.011179,1.316582,0.021663,0.636713
3270,0.011204,1.311660,0.021724,0.633672
3271,0.011376,1.287523,0.021740,0.630987
